In [2]:
import re
"https://docs.google.com/spreadsheets/d/1OReP9PTU8p6VNrGRX1ePl0D6h9mhDMy1bO8B8pRjhzM/edit?usp=sharing"
def convert_google_sheet_url(url):
    # Regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'

    # Replace function to construct the new URL for CSV export
    # If gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'

    # Replace using regex
    new_url = re.sub(pattern, replacement, url)

    return new_url

# Example usage:
url = 'https://docs.google.com/spreadsheets/d/1OReP9PTU8p6VNrGRX1ePl0D6h9mhDMy1bO8B8pRjhzM/edit?usp=sharing'
new_url = convert_google_sheet_url(url)


In [3]:
import pandas as pd

print(new_url)
# https://docs.google.com/spreadsheets/d/1mSEJtzy5L0nuIMRlY9rYdC5s899Ptu2gdMJcIalr5pg/export?gid=1606352415&format=csv

https://docs.google.com/spreadsheets/d/1OReP9PTU8p6VNrGRX1ePl0D6h9mhDMy1bO8B8pRjhzM/export?format=csv


In [4]:
from langchain_community.document_loaders import DataFrameLoader


df = pd.read_csv(new_url)
df['text'] = df['Question'] + ': ' + df['Answer']
loader = DataFrameLoader(df, page_content_column='text')
documents = loader.load()
documents


[Document(page_content='Hoeveel jaar garantie heeft de stoel: de stoel heeft 5 jaar garantie', metadata={'Product': 'Stane Classic Basic', 'EAN': 8719327684375, 'Question': 'Hoeveel jaar garantie heeft de stoel', 'Answer': 'de stoel heeft 5 jaar garantie'}),
 Document(page_content='Hoeveel jaar garantie heeft de plus versie?: De plus versie heeft 6 jaar garantie', metadata={'Product': 'Stane Classic Plus', 'EAN': 8719327684337, 'Question': 'Hoeveel jaar garantie heeft de plus versie?', 'Answer': 'De plus versie heeft 6 jaar garantie'}),
 Document(page_content="What's price of baseball bat: 20$", metadata={'Product': 'baseball bat', 'EAN': 1231024012041014, 'Question': "What's price of baseball bat", 'Answer': '20$'}),
 Document(page_content="what's material of baseball bat: wood", metadata={'Product': 'baseball bat', 'EAN': 1231024012041014, 'Question': "what's material of baseball bat", 'Answer': 'wood'}),
 Document(page_content='how many types of baseball bat: 3', metadata={'Product'

In [5]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, 
                           embedding_function, 
                           persist_directory=persist_directory)

In [7]:



query = "price Nike Shoes?"
docs = db.max_marginal_relevance_search(query,k=2, fetch_k=3)
docs


[Document(page_content="What's the price of pair of Nike shoes?: 10$", metadata={'Answer': '10$', 'EAN': 1231231312, 'Product': 'Nike Shoes', 'Question': "What's the price of pair of Nike shoes?"}),
 Document(page_content="What's material of Nike Shoes?: Wool", metadata={'Answer': 'Wool', 'EAN': 1231231312, 'Product': 'Nike Shoes', 'Question': "What's material of Nike Shoes?"})]

In [8]:
retriever = db.as_retriever(search_type='mmr', k=2, fetch_k=3)

In [33]:
from langchain.chat_models import ChatOpenAI
openai_api_key=""

llm = ChatOpenAI(
        temperature=0.5, model="gpt-4", openai_api_key=openai_api_key, verbose=True
    )

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
PREFIX="""You are a product analyst that will give the answer based on the provided context only. 
Please reply with the user's language. If user speaks Dutch, reply with Dutch
If you cannot find the information, must reply I don't know in the user's language."""

template = PREFIX + """
The context provided:
{context}

Question: {question}
Answer:
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context" ,"question"])

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_type='mmr', k=3, fetch_k=5),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT, "verbose": True},
    verbose=True,
)
result = qa_chain({"query": query})
result["result"]

In [34]:
db.as_retriever(search_type='mmr', k=3, fetch_k=5)

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f8f0627fb50>, search_type='mmr')

In [35]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
PREFIX="""You are a product analyst that will give the answer based on the provided context only. 
Please reply with the user's language. If user speaks Dutch, reply with Dutch
If you cannot find the information, must reply I don't know in the user's language."""

template = PREFIX + """
The context provided:
{context}

Question: {question}
Answer:
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context" ,"question"])

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_type='mmr', k=3, fetch_k=5),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT, "verbose": True},
    verbose=True,
)
result = qa_chain({"query": query})
result["result"]

In [41]:
query = "Hoeveel jaar garantie heeft de plus versie?"

Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a product analyst that will give the answer based on the provided context only. 
Please reply with the user's language. If user speaks Dutch, reply with Dutch
If you cannot find the information, must reply I don't know in the user's language.
The context provided:
What's the price of pair of Nike shoes?: 10$

What's material of Nike Shoes?: Wool

Hoeveel jaar garantie heeft de stoel: de stoel heeft 5 jaar garantie

how many types of baseball bat: 3

Question: gía giày adidas?
Answer:


> Finished chain.

> Finished chain.

> Finished chain.


'Tôi không biết'

In [ ]:

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = OpenAI()

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)

/Users/Jimmy/miniconda3/envs/deep-learning-ws/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


' Justin Beiber was born on March 1, 1994. The Super Bowl is typically played in early February, so we need to look at the Super Bowl that occurred in 1994. \n\nIn 1994, the Super Bowl was Super Bowl XXVIII, which was played on January 30, 1994. The teams that played in this Super Bowl were the Dallas Cowboys and the Buffalo Bills. \n\nThe Dallas Cowboys won the Super Bowl, defeating the Buffalo Bills by a score of 30-13. Therefore, the Dallas Cowboys won the Super Bowl in the year Justin Beiber was born.'

In [ ]:
from langchain_openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent, create_csv_agent
from langchain_openai import ChatOpenAI

PREFIX="You are a product analyst with the information given in the sheet below. You will give the answer based on the information in the sheet only. If you cannot find the information, say I don't know."

agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    prefix=PREFIX,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

agent_csv = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    new_url,
    prefix=PREFIX,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [ ]:
agent_csv.run("Hoeveel jaar garantie heeft ean 8719327684337?")



> Entering new AgentExecutor chain...
De EAN 8719327684337 heeft 6 jaar garantie.

> Finished chain.


'De EAN 8719327684337 heeft 6 jaar garantie.'

In [ ]:
agent.run("Hoeveel jaar garantie heeft de plus versie?")





> Entering new AgentExecutor chain...
De plus versie heeft 6 jaar garantie.

> Finished chain.


'De plus versie heeft 6 jaar garantie.'